In [12]:
import json
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [38]:
with open('labels.json', 'r') as json_file:
    label_to_number  = json.load(json_file)

print(label_to_number )


{'Normal': 0, 'Bundle bra': 1, 'Supraventr': 2, 'Ventricula': 3, 'escape': 4}


In [ ]:
data = np.load('data_tranning.npy', allow_pickle=True)

array([([[-0.145, -0.065], [-0.145, -0.065], [-0.145, -0.065], ..., [-0.39 , -0.295], [-0.4  , -0.29 ], [-0.405, -0.285]], 'Supraventr'),
       ([[-0.535, -0.05 ], [-0.505,  0.085], [-0.415,  0.195], ..., [-0.365, -0.255], [-0.375, -0.265], [-0.375, -0.26 ]], 'Supraventr'),
       ([[-0.39 , -0.275], [-0.395, -0.265], [-0.39 , -0.285], ..., [-0.425, -0.4  ], [-0.43 , -0.42 ], [-0.42 , -0.4  ]], 'Normal'),
       ...,
       ([[-0.22 ,  0.01 ], [-0.205,  0.03 ], [-0.215,  0.035], ..., [-0.42 ,  0.005], [-0.41 , -0.01 ], [-0.395, -0.025]], 'Normal'),
       ([[-0.2  ,  0.185], [-0.215,  0.165], [-0.21 ,  0.17 ], ..., [-0.235,  0.16 ], [-0.24 ,  0.165], [-0.25 ,  0.155]], 'Normal'),
       ([[-0.395, -0.015], [-0.4  , -0.005], [-0.42 , -0.015], ..., [-0.395,  0.005], [-0.38 , -0.01 ], [-0.385,  0.01 ]], 'Normal')],
      shape=(16006,), dtype=[('segment', '<f4', (3600, 2)), ('label', '<U10')])

In [37]:
segments = data['segment']  
labels = data['label']  
np.unique(labels)

array(['Bundle bra', 'Normal', 'Supraventr', 'Ventricula', 'escape'],
      dtype='<U10')

In [39]:
labels = [label_to_number.get(label, -1) for label in labels]

In [40]:
labels


[2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,


In [41]:
X_train, X_test, y_train, y_test = train_test_split(segments, labels, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

In [42]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [43]:
class ECGDataset(Dataset):
    def __init__(self, segments, labels):
        self.segments = segments
        self.labels = labels

    def __len__(self):
        return len(self.segments)

    def __getitem__(self, idx):
        return self.segments[idx], self.labels[idx]

# Tạo Dataset và DataLoader
train_dataset = ECGDataset(X_train, y_train)
val_dataset = ECGDataset(X_val, y_val)
test_dataset = ECGDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [57]:
# Duyệt 1 batch từ train_loader
for inputs, labels in train_loader:
    print(f"Input shape: {inputs.shape}")
    print(f"Label shape: {labels.shape}")
    break  # Chỉ lấy batch đầu tiên


Input shape: torch.Size([64, 3600, 2])
Label shape: torch.Size([64])


In [61]:
import torch.nn as nn
import torch.nn.functional as F

class ECG_CNN(nn.Module):
    def __init__(self, num_classes):
        super(ECG_CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=7, padding=3)
        self.bn1 = nn.BatchNorm1d(16)
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(64)
        self.pool3 = nn.MaxPool1d(kernel_size=2)

        self.global_pool = nn.AdaptiveAvgPool1d(1)  # Tự động thu nhỏ chiều L về 1
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.global_pool(x).squeeze(-1)  # (B, 64)
        x = self.fc(x)
        return x


In [62]:
model = ECG_CNN(5)

In [63]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [67]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in train_loader:
            # inputs = inputs.unsqueeze(1)  
            inputs = inputs.permute(0, 2, 1) 
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            
            predicted = outputs.argmax(dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.permute(0, 2, 1) 
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                predicted = outputs.argmax(dim=1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total

        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.2f}%, "
              f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100)


Epoch [1/100], Train Loss: 0.4134, Train Accuracy: 85.40%, Validation Loss: 0.5197, Validation Accuracy: 82.30%
Epoch [2/100], Train Loss: 0.4012, Train Accuracy: 86.09%, Validation Loss: 0.4473, Validation Accuracy: 84.56%
Epoch [3/100], Train Loss: 0.3917, Train Accuracy: 86.06%, Validation Loss: 0.4784, Validation Accuracy: 81.92%
Epoch [4/100], Train Loss: 0.3892, Train Accuracy: 86.15%, Validation Loss: 0.6870, Validation Accuracy: 75.82%
Epoch [5/100], Train Loss: 0.3785, Train Accuracy: 86.62%, Validation Loss: 0.4882, Validation Accuracy: 81.59%
Epoch [6/100], Train Loss: 0.3684, Train Accuracy: 86.64%, Validation Loss: 0.3993, Validation Accuracy: 85.87%
Epoch [7/100], Train Loss: 0.3538, Train Accuracy: 87.67%, Validation Loss: 0.4180, Validation Accuracy: 84.74%
Epoch [8/100], Train Loss: 0.3431, Train Accuracy: 87.62%, Validation Loss: 0.3643, Validation Accuracy: 86.73%
Epoch [9/100], Train Loss: 0.3462, Train Accuracy: 87.45%, Validation Loss: 0.3970, Validation Accuracy: